# Computes Fragments from the PDB Fragment File

First select which data set you would like to generate.

In [1]:
import ipywidgets as widgets
from glob import glob
from os.path import basename, splitext

datasets = [basename(x) for x in glob("Geometry/*")]
w = widgets.Dropdown(
    options=datasets,
    description='Data Set:',
    disabled=False,
)
display(w)

Dropdown(description=u'Data Set:', options=('TimeStep', 'Size'), value='TimeStep')

## Define the Input Files

In [2]:
namelist = []
dataset = w.value
for f in glob("Geometry/"+dataset+"/*.pdb"):
    namelist.append(splitext(basename(f))[0])

## Read the input

In [3]:
frag_indices = {}
for f in namelist:
    frag_indices[f] = []
    with open("Geometry/"+dataset+"/"+f+".pdb", 'r') as ifile:
        for line in ifile:
            split = line.split()
            if len(split) > 1 and split[0] == "ATOM":
                frag_indices[f].append(split[3]+"-"+split[4])

## Build the dictionary

In [4]:
frag_dict = {}
for f in namelist:
    frag_dict[f] = {}
    for i in range(0,len(frag_indices[f])):
        key = frag_indices[f][i]
        if key in frag_dict[f]:
            frag_dict[f][key].append(i+1)
        else:
            frag_dict[f][key] = [i+1]

## Write To Yaml File

In [6]:
for f in namelist:
    with open("FragmentInput/"+dataset+"/"+f+"-frag.yaml", 'w') as ofile:
        for key, value in frag_dict[f].items():
            ofile.write("- "+str(value)+"\n")